In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black
# notebook setup

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())

In [ ]:
# model_name_l2 =
# model_name_h1 =  

In [ ]:
# load the data
import numpy as np

path = 'rawdata/raw_data/VDP_beta_0.1_grid_30x30.npy'
data = np.load(path)
data.dtype

In [ ]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

### Test the PDPA algorithm

In [ ]:
import torch
from scr.PDPA import PDPA

# pdpa_1 = PDPA(
#     data = data_dict,
#     alpha = 1e-3, 
#     gamma = 5.0,
#     power = 2.1,
#     activation = torch.relu,
#     loss_weights = [1.0, 0.0],
#     verbose=True
# )

# best_iteration_1 = pdpa_1.retrain(
#     num_iterations = 40, 
#     num_insertion= 1, 
#     threshold = 1e-10,
#     )

In [ ]:
# best_iteration_1

## Test effect of different gamma with increased neurons added

In [ ]:
gammas = [10.0, 5.0, 1.0, 1e-1, 1e-2]
alpha = 1e-3
power = 2.1
loss_weight_h1 = [1.0, 1.0]
loss_weight_l2 = [1.0, 0.0]

num_iterations = 40
num_insertions = 1
pruning_threshold = 1e-10

In [ ]:
pdpa_list_l2 = []
best_iteration_l2 = []
for gamma in gammas:
    pdpa = PDPA(
        data=data_dict,
        alpha=alpha,
        gamma=gamma,
        power=power,
        activation=torch.relu,
        loss_weights=loss_weight_l2,
        verbose=False,
    )

    best_iteration = pdpa.retrain(
    num_iterations = num_iterations, 
    num_insertion= num_insertions, 
    threshold = pruning_threshold,
    verbose=False
    )

    pdpa_list_l2.append(pdpa)
    best_iteration_l2.append(best_iteration)


In [ ]:
best_iteration_l2

In [ ]:
pdpa_list_h1 = []
best_iteration_h1 = []

for gamma in gammas:
    pdpa = PDPA(
        data=data_dict,
        alpha=alpha,
        gamma=gamma,
        power=power,
        activation=torch.relu,
        loss_weights=loss_weight_h1,
        verbose=False,
    )

    best_iteration = pdpa.retrain(
    num_iterations = num_iterations, 
    num_insertion= num_insertions, 
    threshold = pruning_threshold,
    verbose=False
    )

    pdpa_list_h1.append(pdpa)
    best_iteration_h1.append(best_iteration)

In [ ]:
best_iteration_h1

In [ ]:
import pickle

model_l2 = {
    "gammas": gammas,
    "pdpa_list_l2": pdpa_list_l2,
    "alpha": alpha,
    "power": power,
    "num_iteration": num_insertions,
    "num_insertion": num_insertions,
    "pruning_threshold": pruning_threshold,
    "best_iteration_l2": best_iteration_l2
}

model_h1 = {
    "gammas": gammas,
    "pdpa_list_h1": pdpa_list_h1,
    "alpha": alpha,
    "power": power,
    "num_iteration": num_insertions,
    "num_insertion": num_insertions,
    "pruning_threshold": pruning_threshold,
    "best_iteration_h1": best_iteration_h1
}


with open("models/pdpa_vdp_model_l2_3.pkl", "wb") as f:
    pickle.dump(model_l2, f)

with open("models/pdpa_vdp_model_h1_3.pkl", "wb") as f:
    pickle.dump(model_h1, f)

In each iteration we tried to add 5 neurons. The following plots indicates the relation of loss & number of neurons. 

In [ ]:
from scr.metric import plot_pdpa_val_loss_histories_by_gamma

plot_pdpa_val_loss_histories_by_gamma(model_h1)

In [ ]:
plot_pdpa_val_loss_histories_by_gamma(model_l2, pdpa_key="pdpa_list_l2")